In [ ]:
# ----------------------------------------
# Python Script to web scrap validation dataset from NCDHHS E. coli testing results
# Author: Jennifer Zhang
# Last edited: 2025-07-23
# ----------------------------------------

import requests
from bs4 import BeautifulSoup
import pdfplumber
import fitz
import re
import csv
import os
from geocodio import GeocodioClient
import pandas as pd

In [ ]:
# Fetch and parse the NCDHHS Hurricane Helene test results website
base_url = 'https://celr.dph.ncdhhs.gov/hurricaneHelene'
response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find county IDs
county_IDs = []
counties = soup.find('select', id='client.rasclientId')
for county in counties.find_all('option'):
    id = county['value']
    if len(id) > 0: # Ignores ***Select*** option's empty value
        id = id.replace(" ", "+")
        county_IDs.append(id)
    # print(county.text.strip())
# print(county_IDs)

with open("county_ids_hurricane.txt", "w") as f:
    for county_id in county_IDs:
        f.write(f"{county_id}\n")

In [ ]:
# Loop through county pages and scrape PDF links
county_IDs = open("county_ids_hurricane.txt").read().splitlines()
pdf_links = []

for county_id in county_IDs:
    url = f"{base_url}?client.rasclientId={county_id}&filterBy=1&recentDay=5&docFrom=&docTo="
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    if not soup.find_all('div', class_='text-error'):
        for sample in soup.find_all('a', href=True, target='_blank'):
            pdf_links.append(sample['href'])

with open("pdf_links_hurricane.txt", "w") as f:
    for link in pdf_links:
        f.write(f"{link}\n")

In [ ]:
def extract_text_from_pdf(pdf_path):
    # Parse the PDF
    with pdfplumber.open(pdf_path) as pdf:
        text = pdf.pages[0].extract_text()

    doc = fitz.open("temp.pdf")

    data = {
        "System_Name": re.search(r"Name of System:\s*([A-Z0-9\s&.,\-]+)", text), # get rid of non-all caps text afterwards
        "Street_Address": None,  # Placeholders for address
        "City": None,
        "State": None,
        "Zip": None,
        "EIN": re.search(r"EIN:\s?(\S+)", text),
        "StarLiMS_ID": re.search(r"StarLiMS ID:\s*(\S+)", text),
        "Date_Collected": re.search(r"Date Collected:\s*(\d{2}/\d{2}/\d{4})", text),
        "Time_Collected": re.search(r"Time Collected:\s*(\d{2}:\d{2})", text),
        "Date_Received": re.search(r"Date Received:\s*(\d{2}/\d{2}/\d{4})", text),
        "Time_Received": re.search(r"Time Received:\s*(\d{2}:\d{2})", text),
        "Sampling_Point": re.search(r"Sampling Point:\s*(.*?)\n", text),
        "Sample_Type": re.search(r"Sample Type:\s*(Treated|[\w\s]+?)\s+GPS No\.", text),
        "Treatment": re.search(r"Treatment:\s*(Chlorinated|[\w\s]+?)\s+Well Permit No\.", text),
        "Days_Since_Treatment": re.search(r"Days Since Chlorine Treatment\s*(.*?)\n", text),
        # "Days_Since_Treatment": re.search(r"Days Since Chlorine Treatment\s*(Unknown/Not Given|<= 7 Days|>7 Days)\n", text),
        "Comment": re.search(r"Comment:\s*(.*?)\nColi", text),
        "Colilert_Method": re.search(r"Method:\s*(.*?)\n", text),
        "Total_Coliform_Result": re.search(r"Total Coliform\s+(Absent|Present|Not Tested|[0-9<>.,]+)(?=\s+)", text),
        "Total_Coliform_Date": re.search(r"Total Coliform\s+\w+\s+(\d{2}/\d{2}/\d{4})", text),
        "Ecoli_Result": re.search(r"E\. coli\s+(Absent|Present|Not Tested|[0-9<>.,]+)(?=\s+)", text),
        "Ecoli_Date": re.search(r"E\. coli\s+\w+\s+(\d{2}/\d{2}/\d{4})", text),
    }

    # Clean up and finalize regex values
    cleaned_data = {k: (v.group(1).strip() if v else "") for k, v in data.items()}

    # Add address extraction logic
    street_line = None
    city_line = None

    # Define tight coordinate ranges with a little tolerance
    x_range = (415, 420)
    street_y_range = (158, 162)
    city_y_range = (189, 192)

    for page in doc:
        blocks = page.get_text("blocks")

        for block in blocks:
            x0 = block[0]
            y0 = block[1]
            text = block[4].strip()

            if x_range[0] <= x0 <= x_range[1] and street_y_range[0] <= y0 <= street_y_range[1]:
                street_line = text

            elif x_range[0] <= x0 <= x_range[1] and city_y_range[0] <= y0 <= city_y_range[1]:
                city_line = text

    if street_line and city_line:
        # Extract street only (remove possible names after the address)
        street = street_line.strip().split('\n', 1)[0]
        if street:
            cleaned_data["Street_Address"] = street
        else:
            cleaned_data["Street_Address"] = ""
            print(f"Warning: Street address not found for {cleaned_data['StarLiMS_ID']}")

        # Extract city, state, zip
        city_match = re.search(r'([A-Za-z\s]+),\s+([A-Z]{2})\s+(\d{5})', city_line)        
        
        if city_match:
            city, state, zip_code = city_match.groups()
            cleaned_data["City"] = city.strip()
            cleaned_data["State"] = state
            cleaned_data["Zip"] = zip_code
        else:
            cleaned_data["City"] = ""
            cleaned_data["State"] = ""
            cleaned_data["Zip"] = ""
            print(f"Warning: City, state, or zip not found for {cleaned_data['StarLiMS_ID']}")

    return cleaned_data

Test extract_text()

In [ ]:
link = 'showReportPublic?folderNo=ES250211-0076&org=v11'
url = f"https://celr.dph.ncdhhs.gov/{link}"
pdf_response = requests.get(url)
with open("temp.pdf", "wb") as f:
    f.write(pdf_response.content)

with pdfplumber.open("temp.pdf") as pdf:
    text = pdf.pages[0].extract_text()
# print(text)

extracted_text = extract_text_from_pdf(pdf_path="temp.pdf")
# print(extracted_text)

4312 District Drive
North Carolina State Laboratory of Public Health MSC 1918
Raleigh, NC 27699-1918
Environmental Sciences
http://slph.ncpublichealth.com
Phone: 919-733-7308
Microbiology
Fax: 919-715-8611
Certificate of Analysis
FINAL REPORT
Report to: Jason Pierce Name of System:
ALLEGHANY CO ENVIRONMENTAL HEALTH Doug Cox
PO BOX 309 3490 Chestnut Grove
Sparta, NC 28675 Sparta, NC 28675
EIN: 566001534EH Delivery: NC Courier Alleghany County
StarLiMS ID: ES250211-0076 Date Collected: 02/10/2025 Time Collected: 09:30 By: Doug Cox
Date Received: 02/11/2025 Time Received: 08:02 By: Julie Schiavone
Barcode ID: 000714
Sample Source: Hurricane Helene Sampling Point: Bathroom sink
Sample Type: GPS No.
Treatment: Well Permit No.
Days Since Chlorine Treatment Unknown/Not Given
Comment: Water source: Spring
Colilert - Quanti-Tray2000 Profile Method: SM 9223B
Analyte Test Result Unit Conclusion Date Tested
Total Coliform 16 MPN/100mL 02/11/2025
E. coli <1 MPN/100mL 02/11/2025
Report Date: 02/12/2

In [ ]:
all_data = []
pdf_links = open("pdf_links_hurricane.txt").read().splitlines()

# Loop through the PDF links
for link in pdf_links:
    print(f"Processing {link}...")
    
    # Download the PDFs
    url = f"https://celr.dph.ncdhhs.gov/{link}"
    pdf_response = requests.get(url)
    with open("temp.pdf", "wb") as f:
        f.write(pdf_response.content)

    # Extract text from the PDF
    extracted_text = extract_text_from_pdf("temp.pdf")
    os.remove("temp.pdf")

    all_data.append(extracted_text)

Processing showReportPublic?folderNo=ES250430-0059&org=v11...
Processing showReportPublic?folderNo=ES250415-0053&org=v11...
Processing showReportPublic?folderNo=ES250327-0059&org=v11...
Processing showReportPublic?folderNo=ES250327-0060&org=v11...
Processing showReportPublic?folderNo=ES250325-0047&org=v11...
Processing showReportPublic?folderNo=ES250304-0034&org=v11...
Processing showReportPublic?folderNo=ES250226-0050&org=v11...
Processing showReportPublic?folderNo=ES250219-0113&org=v11...
Processing showReportPublic?folderNo=ES250211-0076&org=v11...
Processing showReportPublic?folderNo=ES250130-0094&org=v11...
Processing showReportPublic?folderNo=ES250130-0092&org=v11...
Processing showReportPublic?folderNo=ES250130-0093&org=v11...
Processing showReportPublic?folderNo=ES250116-0041&org=v11...
Processing showReportPublic?folderNo=ES241219-0057&org=v11...
Processing showReportPublic?folderNo=ES241217-0050&org=v11...
Processing showReportPublic?folderNo=ES241211-0080&org=v11...
Processi

In [ ]:
# Write to CSV
with open("ecoli_report_hurricane.csv", mode="w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=extracted_text.keys())
    writer.writeheader()
    for sample in all_data:
        writer.writerow(sample)

print("ecoli_report_hurricane.csv created successfully")

ecoli_report.csv created successfully



Manually edited **ecoli_report_hurricane.csv** to create **ecoli_clean_hurricane.csv**. The edits are described in the notes below.

Notes:
- Corrected System_Name by removing extra capitalized letter at the end of the string
- Deleted irrelevant columns: Date_Received, Time_Received, Colilert Method, Total_Coliform_Date, and Ecoli_Date
- Deleted samples with treated water -> also deleted Treatment and Days_Since_Treatment columns
- Added Street_Address_Concat for geocoding which concatenates Street_Address, City, State, and Zip columns
- Entered County, State, and Zip data for '56 Fox Hill Rd' (StarLiMS_ID = ES250108-0061)

In [ ]:
# Set up Geocodio client
GEOCODIO_API_KEY = '7001d7bec0fe8e04c07b84e6ec76664b0876076'
client = GeocodioClient(GEOCODIO_API_KEY)

# Forward geocode the Street_Address_Concat column from 'ecoli_clean_hurricane.csv' 
df = pd.read_csv('ecoli_clean_hurricane.csv')
street_addresses = df['Street_Address_Concat'].tolist()
# print(street_addresses)

geocoded = client.batch_geocode(street_addresses)
lat = []
lng = []
accuracy = []
accuracy_type = []

for i, result in enumerate(geocoded):
    lat.append(result['results'][0]['location']['lat'])
    lng.append(result['results'][0]['location']['lng'])
    accuracy.append(result['results'][0]['accuracy'])
    accuracy_type.append(result['results'][0]['accuracy_type'])

    # Geocod.io accuracy score documentation: https://www.geocod.io/docs/?python#accuracy-score
    if result['results'][0]['accuracy'] < 0.8:
        print(f"Warning: Geocoding accuracy for {street_addresses[i]} is {result['results'][0]['accuracy']}. Accuracy type is {result['results'][0]['accuracy_type']}.")

# Write the geocoded coordinates (latitude, longitude) to 'ecoli_coords_hurricane.csv' file
df['Latitude'] = lat
df['Longitude'] = lng
df['Accuracy'] = accuracy
df['Accuracy_Type'] = accuracy_type

df.to_csv('ecoli_coords_hurricane.csv', index=False)